# Web Scraping

###  *******Skip to <ins>***Get All Games List page links***</ins> halfway down to see the actual scraping***

## Initial Web Scrape testing

In [3]:
import requests
import sys
import re
from bs4 import BeautifulSoup as bs
import pandas as pd, numpy as np
import warnings
warnings.filterwarnings(action='once')
from icecream import ic
from random import randint, random
from time import sleep
from timeit import default_timer as timer
from playsound import playsound
from yag import yaggit
from concurrent import futures
from itertools import count

In [638]:
base = "https://www.metacritic.com"
# first_page = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed"
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0'}
# review_dict = {'name':[], 'date':[], 'game':[], 'rating':[], 'review':[]}
# game_dict = {'title':[], 'release_date':[], 'genre':[], 'platforms':[], 'developer':[], 'esrb_rating':[], 'ESRBs':[], \
#              'metascore':[], 'userscore':[],'critic_reviews':[], 'user_reviews':[], 'num_players':[], 'summary':[]}

In [3]:
def email(message):
    yaggit(to='trentenmanugas@gmail.com', subject='Scraper', message = message)

In [4]:
def get_details(soup, text, col, idx=1):
    details = []
    if soup.find('th', scope='row', text = text)==None:
        game_dict[col].append(None)
        return
    for i in soup.find('th', scope='row', text = text).next_siblings:
        details.append(i.text)
    details_ = ' '.join(details[idx].strip().split())
    game_dict[col].append(details_)

In [5]:
def get_platforms(soup):
    if soup.find('span', class_="label", string = "Also On:")==None:
        game_dict['platforms'].append(soup.find('span', class_="platform").text.strip())
    else:
        consoles = [soup.find('span', class_="platform").text.strip()]
        other_consoles = [console.text  for console in soup.find_all('a', class_="hover_none")[1:]]
        consoles.extend(other_consoles)
        game_dict['platforms'].append(', '.join(consoles))

In [6]:
def get_user_reviews(soup):
    if soup.find_all('span', class_ = 'count')[1].find('a')!=None:
        game_dict['user_reviews'].append(soup.find_all('span', class_ = 'count')[1].find('a').text[:-8])
    else:
        game_dict['user_reviews'].append(None)

In [7]:
def get_numplayers(soup):
    P = soup.find(text = 'Number of Players:')
    O = soup.find(text = 'Number of Online Players:')
    if P==None:
        if O==None or O.next_element.text=='No Online Multiplayer':
            game_dict['num_players'].append('1 Player')
            return
        else:
            game_dict['num_players'].append(O.next_element.text)
            return
    else:
        if O==None or O.next_element.text=='No Online Multiplayer':
            game_dict['num_players'].append(P.next_element.text)
            return
        else:
            players = [P.next_element.text, O.next_element.text]
            game_dict['num_players'].append(', '.join(players))
            return

In [8]:
def get_summary(soup):
    if soup.find('span', class_="label", string = "Also On:")!=None:
        if len(soup.find_all('span', class_ = 'data'))==4:
            game_dict['summary'].append(soup.find_all('span', class_ = 'data')[3].text)
        else:
            game_dict['summary'].append(None)
    else:
        if len(soup.find_all('span', class_ = 'data'))==3:
            game_dict['summary'].append(soup.find_all('span', class_ = 'data')[2].text)
        else:
            game_dict['summary'].append(None)

In [506]:
def parse_info(details_soup, details_link):
    # Get to Details Page
    
    count = 1 
    while details_soup.a==None and count<3:
        print('Error, Trying DETAILS Again')
        sleep(10)
        details_response = requests.get(details_link+'/robots.text', headers = user_agent)
        details_soup = bs(details_response.text, 'html.parser')
        count += 1
        print('Still None?:', details_soup.a==None)
        print('response code:', details_response.status_code)
    try:
        for detail in details_soup.find_all('div', id='main'):
            detail.find('h1').text
            detail.find_all('span', class_='data')[1].text
            detail.find('span', class_="platform").text.strip()
            detail.find('span', itemprop='ratingValue').text
            detail.find_all('a', class_='metascore_anchor')[1].contents[1].text
    except Exception as e:
        print('\n',sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print('Info Failed')
        playsound('ding.mp3')
#         email('Details Failed')
        return 'done'
        
#     Get game Details
    try:
        for detail in details_soup.find_all('div', id='main'):
            game_dict['title'].append(detail.find('h1').text)
            game_dict['release_date'].append(detail.find_all('span', class_='data')[1].text)
            get_details(detail, 'Genre(s):', 'genre')
    #         get_platforms(detail)
            game_dict['platforms'].append(detail.find('span', class_="platform").text.strip())
            get_details(detail, 'Developer:', 'developer')
            get_details(detail, 'Rating:', 'esrb_rating')
            get_details(detail, 'ESRB Descriptors:', 'ESRBs', idx=0)
            game_dict['metascore'].append(detail.find('span', itemprop='ratingValue').text)
            game_dict['userscore'].append(detail.find_all('a', class_='metascore_anchor')[1].contents[1].text)
            game_dict['critic_reviews'].append([i.find_all('span') for i in 
                                                detail.find_all('span', class_='count')][0][2].text.strip())
            get_user_reviews(detail)
            get_numplayers(detail)
            get_summary(detail)
    except Exception as e:
        print('\n',sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print('Info Failed')
        playsound('ding.mp3')
        email('Details Failed')
        return 'done'

In [591]:
def parse_reviews(user_review_soup, user_review_link):
        
    count = 1
    while user_review_soup.a==None and count<3:
        print('Error, Trying REVIEWS Again')
        sleep(10)
        user_review_response = requests.get(user_review_link+'/robots.text', headers = user_agent)
        user_review_soup = bs(user_review_response.text, 'html.parser')
        count += 1
        print('Still None?:', user_review_soup.a==None)
        print('response code:', user_review_response.status_code)
        
    try:
        for review in user_review_soup.find_all('div', class_='review_content'):
            if review.find('div', class_='name') == None:
                break
    #                 get_consoles(user_review_soup) 
            user_review_soup.find('h1').text
            review.find('div', class_='name').find(recursive = True).text
            review.find('div', class_='date').text
            review.find('div', class_='review_grade').find_all('div')[0].text
            if review.find('span', class_='blurb blurb_expanded'):
                review.find('span', class_='blurb blurb_expanded').text
            elif review.find('div', class_='review_body').find('span')==None:
                continue
            else:
                review.find('div', class_='review_body').find('span').text
                
    except Exception as e:
        print('\n',sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print('Reviews Failed')
        playsound('ding.mp3')
        email('Reviews Failed')
        return 'done'
        

#           credit: Adeline Ong
#           https://towardsdatascience.com/web-scraping-metacritic-reviews-using-beautifulsoup-63801bbe200e

#             Loop through User Review Page and Extract Review Data into a dictionary
    try:
        for review in user_review_soup.find_all('div', class_='review_content'):
            if review.find('div', class_='name') == None:
                break
    #                 get_consoles(user_review_soup) 
            review_dict['game'].append(user_review_soup.find('h1').text)
            review_dict['name'].append(review.find('div', class_='name').find(recursive = True).text)
            review_dict['date'].append(review.find('div', class_='date').text)
            review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
            if review.find('span', class_='blurb blurb_expanded'):
                review_dict['review'].append(review.find('span', class_='blurb blurb_expanded').text)
            elif review.find('div', class_='review_body').find('span')==None:
                review_dict['review'].append(None)
            else:
                review_dict['review'].append(review.find('div', class_='review_body').find('span').text)
    except Exception as e:
        print('\n',sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print('Reviews Failed')
        playsound('ding.mp3')
        email('Reviews Failed')
        return 'done'

In [ ]:
def parse_games(startpage_link, next_page, end_page, game_num_start, game_num_end = 101):  
    next_page = next_page
    
#   request from current page
    try:
        response = requests.get(startpage_link, headers = user_agent)
        soup = bs(response.text, 'html.parser')
    except Exception as e:
        print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print(f'Trying page {next_page-1} again')
        sleep(10)
        response = requests.get(startpage_link, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        
        
    count = 1
    while soup.a==None and count<3:
        print('Error, Trying PAGE Again')
        sleep(10)
        response = requests.get(startpage_link, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        count += 1
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
    if soup.a==None:
        print('Trying PAGE last time')
        sleep(15)
        response = requests.get(startpage_link+'/robots.text', headers = user_agent)
        soup = bs(response.text, 'html.parser')
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
        if soup.find('a')==None:
            print(f'{next_page-1} PAGE Failed')
            playsound('ding.mp3')
            email(f'Failed to Connect to Page {next_page-1}')
            return
            
    
#     Loop through current page and extract links to game pages
    current_page = int(soup.find('span', class_='page_num').text)
    print(f'parsing page: {current_page}/{end_page}')
    game_num = game_num_start
    for game in soup.find_all('a', attrs = {'class':'title'})[game_num - 1:game_num_end]:
        game_link = url + game.get('href')
        
#         Connects to game page summary
        sleep(3)
        try:
            game_response = requests.get(game_link, headers = user_agent)
            game_soup = bs(game_response.text, 'html.parser')
        except Exception as e:
            print('Failed to connect to game page')
            print('Game response code:', game_response.status_code)
            print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
            print('Trying again...')
            sleep(10)
            game_response = requests.get(game_link, headers = user_agent)
            game_soup = bs(game_response.text, 'html.parser')
            
        count = 1
        while game_soup.a==None and count<3:
            print('Error, Trying GAME Again')
            sleep(10)
            game_response = requests.get(game_link, headers = user_agent)
            game_soup = bs(game_response.text, 'html.parser')
            count += 1
            print('Still None?:', game_soup.a==None)
            print('response code:', game_response.status_code)
        if game_soup.a==None:
            print('Trying GAME last time')
            sleep(15)
            game_response = requests.get(game_link+'/robots.text', headers = user_agent)
            game_soup = bs(game_response.text, 'html.parser')
            print('Still None?:', game_soup.a==None)
            print('response code:', game_response.status_code)
            if game_soup.find('a')==None:
                print(f'Game {game_num} GAME Failed')
                playsound('ding.mp3')
                email(f'Failed to Connect to Game {game_num}')
                return
        print('Making Game Connections...', end = ' ')
        sleep(3)
        
        
        #         Connect to Details Page
        try:
            details_link = game_link + '/details'
#             game_soup.find('a', attrs = {'class':'action'}, text = 'Details & Credits').get('href')
            details_response = requests.get(details_link, headers = user_agent)
            details_soup = bs(details_response.text, 'html.parser')
        except Exception as e:
            sleep(10)
            print('\n',sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
            print('Failed to connect to Details page')
            print('Details response code:', details_response.status_code)
            print('Trying Details again')
            details_response = requests.get(details_link, headers = user_agent)
            details_soup = bs(details_response.text, 'html.parser')
        
        count = 1
        while details_soup.a==None and count<3:
            print('Error, Trying Details Again')
            sleep(10)
            details_response = requests.get(details_link, headers = user_agent)
            details_soup = bs(details_response.text, 'html.parser')
            count += 1
            print('Still None?:', details_soup.a==None)
            print('Details response code:', details_response.status_code)
        if details_soup.find('a')==None:
            print('Trying Details last time')
            sleep(15)
            details_response = requests.get(details_link+'/robots.text', headers = user_agent)
            details_soup = bs(details_response.text, 'html.parser')
            if details_soup.find('a')==None:
                print(f'Game {game_num} Details Failed')
                print('Details response code:', details_response.status_code)
                playsound('ding.mp3')
                email(f'Game {game_num} Details Failed')
                return
            
            
        #             Connects to User Review page
        try:
            sleep(3)
            user_review_link = game_link + '/user-reviews'
#             game_soup.find('a', attrs = {'class':'action'}, text = 'User Reviews').get('href')
            user_review_response = requests.get(user_review_link, headers = user_agent)
            user_review_soup = bs(user_review_response.text, 'html.parser')
        except Exception as e:
            sleep(10)
            print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
            print('Failed to connect to Reviews page')
            print('Reviews response code:', user_review_response.status_code)
            print('Trying Reviews again')
            user_review_response = requests.get(user_review_link, headers = user_agent)
            user_review_soup = bs(user_review_response.text, 'html.parser')
            
            
        count = 1
        while user_review_soup.a==None and count<3:
            print('Error, Trying Reviews Again')
            sleep(10)
            user_review_response = requests.get(user_review_link, headers = user_agent)
            user_review_soup = bs(user_review_response.text, 'html.parser')
            count += 1
            print('Still None?:', user_review_soup.a==None)
            print('Reviews response code:', user_review_response.status_code)
        if user_review_soup.a==None:
            print('Trying Reviews last time')
            sleep(15)
            user_review_response = requests.get(user_review_link+'/robots.text', headers = user_agent)
            user_review_soup = bs(user_review_response.text, 'html.parser')
            print('Still None?:', user_review_soup.a==None)
            print('Reviews response code:', user_review_response.status_code)
            if user_review_soup.a==None:
                print(f'Game {game_num} Reviews Failed')
                playsound('ding.mp3')
                email(f'Game {game_num} Reviews Failed')
                return
        
        
#         Get Game info
        if parse_info(details_soup)=='done':
            return
        
        
#         Get Game Reviews
        if parse_reviews(user_review_soup)=='done':
            return
        
        print(f'Game {game_num} completed')
        game_num +=1
        
    if next_page == (end_page+1):
        find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
        global startpage
        startpage = url + find_next_page.get('href')
        print(f'Done, Next Start Page: {startpage}')
        print('Copy and Paste this link or use startpage variable')
        playsound('ding.mp3')
        email('Completed')
        return startpage_link
    if next_page>(end_page+1):
        print('Increase End page')
        playsound('ding.mp3')
        return
    if current_page>=next_page:
        print('Increase Next page')
        playsound('ding.mp3')
        return
    sleep(10)
    
#     Recurse through the following pages
    find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
    nextpage_link = url + find_next_page.get('href')
    next_page += 1
    parse_games(nextpage_link, next_page, end_page, game_num_start = 1, game_num_end = 101)

In [1]:
page_num = 51
startpage=f'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&page={page_num-1}'

In [ ]:
# start = timer()
# try:
#     parse_games(startpage, next_page = 52, end_page = 54, game_num_start = 1, game_num_end = 101)
# except Exception as e:
#     print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
#     playsound('ding.mp3')
#     email(f'Error {sys.exc_info()[0]} ||| {sys.exc_info()[1]} ||| Error on line {sys.exc_info()[2].tb_lineno}')
    
# end = timer()
# minutes = int((end-start)//60)
# seconds = int((end-start)-(minutes*60))
# print(f'Elapsed Time: {minutes} minutes {seconds} seconds')

In [ ]:
# ['game', 'release_date', 'genre', 'platforms', 'developer', 'esrb_rating', 'ESRBs', 'metascore', 'userscore', 
#  'critic_reviews', 'user_reviews', 'num_players', 'summary'])

In [ ]:
# game_dict = pd.read_csv('metacritic_games_four.csv').drop(['Unnamed: 0'],axis=1).to_dict('list')

In [225]:
def parse_links(startpage_link, next_page, end_page, game_num_start, game_num_end = 101):  
    next_page = next_page
    
#   request from current page
    try:
        response = requests.get(startpage_link, headers = user_agent)
        soup = bs(response.text, 'html.parser')
    except Exception as e:
        print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print(f'Trying page {next_page-1} again')
        sleep(10)
        response = requests.get(startpage_link, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        
        
    count = 1
    while soup.a==None and count<3:
        print('Error, Trying PAGE Again')
        sleep(10)
        response = requests.get(startpage_link, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        count += 1
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
    if soup.a==None:
        print('Trying PAGE last time')
        sleep(15)
        response = requests.get(startpage_link+'/robots.text', headers = user_agent)
        soup = bs(response.text, 'html.parser')
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
        if soup.find('a')==None:
            print(f'{next_page-1} PAGE Failed')
            playsound('ding.mp3')
            email(f'Failed to Connect to Page {next_page-1}')
            return
            
    
#     Loop through current page and extract links to game pages
    current_page = int(soup.find('span', class_='page_num').text)
    print(f'parsing page: {current_page}/{end_page}')
    game_num = game_num_start
    for num in soup.find_all('span', class_="title numbered"):
        links['number'].append(int(num.text.strip()[:-1]))
    for game in soup.find_all('a', attrs = {'class':'title'})[game_num - 1:game_num_end]:
        game_link = url + game.get('href')
        
#         Connects to game page summary
#         sleep(3)
#         try:
#             game_response = requests.get(game_link, headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
#         except Exception as e:
#             print('Failed to connect to game page')
#             print('Game response code:', game_response.status_code)
#             print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
#             print('Trying again...')
#             sleep(10)
#             game_response = requests.get(game_link, headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
            
#         count = 1
#         while game_soup.a==None and count<3:
#             print('Error, Trying GAME Again')
#             sleep(10)
#             game_response = requests.get(game_link, headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
#             count += 1
#             print('Still None?:', game_soup.a==None)
#             print('response code:', game_response.status_code)
#         if game_soup.a==None:
#             print('Trying GAME last time')
#             sleep(15)
#             game_response = requests.get(game_link+'/robots.text', headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
#             print('Still None?:', game_soup.a==None)
#             print('response code:', game_response.status_code)
#             if game_soup.find('a')==None:
#                 print(f'Game {game_num} GAME Failed')
#                 playsound('ding.mp3')
#                 email(f'Failed to Connect to Game {game_num}')
#                 return
#         print('Getting links...', end = ' ')
#         sleep(3)
        
        
        #         Get detail and review links
#         try:
        details_link = game_link + '/details'
        links['detail_links'].append(details_link)
#            
        user_review_link = game_link + '/user-reviews'
        links['review_links'].append(user_review_link)
#             
        
    if next_page == (end_page+1):
        find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
        global startpage
        startpage = url + find_next_page.get('href')
        print(f'Done, Next Start Page: {startpage}')
        print('Copy and Paste this link or use startpage variable')
        playsound('ding.mp3')
#         email('Completed')
        return startpage_link
    if next_page>(end_page+1):
        print('Increase End page')
        playsound('ding.mp3')
        return
    if current_page>=next_page:
        print('Increase Next page')
        playsound('ding.mp3')
        return
    sleep(5)
    
#     Recurse through the following pages
    find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
    nextpage_link = url + find_next_page.get('href')
    next_page += 1
    parse_links(nextpage_link, next_page, end_page, game_num_start = 1, game_num_end = 101)

In [ ]:
# parse_links(startpage, next_page=62, end_page=120, game_num_start = 1, game_num_end = 101) 

In [226]:
page_start = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed"

______

____

_____

# Get All Games List page links

In [78]:
main_first = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed'
# urls = []
# for i in list(range(194)):
#     if i==0:
#         urls.append(main_first)
#     else:
#         urls.append(main+f'&page={i}')

In [8]:
main_pages = pd.DataFrame(urls)
main_pages.head().values

array([['https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed'],
       ['https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&page=1'],
       ['https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&page=2'],
       ['https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&page=3'],
       ['https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&page=4']],
      dtype=object)

In [83]:
# main_pages.to_csv('main_meta_pages.csv')

# Get Details and Review Links

In [99]:
# links = {'number':[], 'detail_links':[], 'review_links':[]}

In [107]:
# iterator = count()

In [108]:
def transform(url):
    page_num = next(iterator)+1
    try:
        response = requests.get(url, headers = user_agent)
        soup = bs(response.text, 'html.parser')
    except Exception as e:
        print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print(f'Trying page again')
        sleep(10)
        response = requests.get(url, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        
        
    count = 1
    while soup.a==None and count<3:
        print('Error, Trying PAGE Again')
        sleep(10)
        response = requests.get(url, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        count += 1
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
    if soup.a==None:
        print('Trying PAGE last time')
        sleep(15)
        response = requests.get(url+'/robots.text', headers = user_agent)
        soup = bs(response.text, 'html.parser')
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
        if soup.find('a')==None:
            print(f'{page_num} PAGE Failed')
            playsound('ding.mp3')
            main_fails.append(url)
#             email(f'Failed to Connect to Page {page_num}')
            return
    
    print(f'parsing page: {page_num}')
    
    for num in soup.find_all('span', class_="title numbered"):
        links['number'].append(int(num.text.strip()[:-1]))
    for game in soup.find_all('a', attrs = {'class':'title'})[:101]:
        game_link = base + game.get('href')
        
        details_link = game_link + '/details'
        links['detail_links'].append(details_link)
        
        user_review_link = game_link + '/user-reviews'
        links['review_links'].append(user_review_link)
    return

In [ ]:
# with futures.ThreadPoolExecutor() as getting:
#     getting.map(transform, urls)

In [11]:
# links_df = pd.DataFrame(links).drop_duplicates().sort_values('number', ascending = True)
links_df.head().values

array([[1,
        'https://www.metacritic.com/game/nintendo-64/the-legend-of-zelda-ocarina-of-time/details',
        'https://www.metacritic.com/game/nintendo-64/the-legend-of-zelda-ocarina-of-time/user-reviews'],
       [2,
        'https://www.metacritic.com/game/playstation/tony-hawks-pro-skater-2/details',
        'https://www.metacritic.com/game/playstation/tony-hawks-pro-skater-2/user-reviews'],
       [3,
        'https://www.metacritic.com/game/playstation-3/grand-theft-auto-iv/details',
        'https://www.metacritic.com/game/playstation-3/grand-theft-auto-iv/user-reviews'],
       [4,
        'https://www.metacritic.com/game/dreamcast/soulcalibur/details',
        'https://www.metacritic.com/game/dreamcast/soulcalibur/user-reviews'],
       [5,
        'https://www.metacritic.com/game/xbox-360/grand-theft-auto-iv/details',
        'https://www.metacritic.com/game/xbox-360/grand-theft-auto-iv/user-reviews']],
      dtype=object)

In [118]:
# links_df.to_csv('links_master.csv')

# Parse Detail Pages

In [121]:
detail_links = links['detail_links']

In [123]:
# iterate = count()

In [136]:
# detail_counts = {'counts':[], 'urls':[]}

In [507]:
def transform_details(url):  
    
    try:
        response = requests.get(url, headers = user_agent)
        soup = bs(response.text, 'html.parser')
    except Exception as e:
        print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print(f'Trying GAME again')
        sleep(10)
        response = requests.get(url, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        
        
    count = 1
    while soup.a==None and count<3:
        print('Error, Trying GAME Again')
        sleep(10)
        response = requests.get(url, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        count += 1
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
    if soup.a==None:
        print('Trying GAME last time')
        sleep(15)
        response = requests.get(url+'/robots.text', headers = user_agent)
        soup = bs(response.text, 'html.parser')
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
        if soup.find('a')==None:
            print(f'GAME Failed')
            failed_detail_links.append(str(url))
            playsound('ding.mp3')
#             email(f'Failed to Connect to Page {page_num}')
            return
    
    num = next(iterate)
    if parse_info(soup, url)=='done':
        print(f'{url} went wrong')

#     lists = [game_dict[key] for key in game_dict.keys()]
#     t = iter(lists)
#     the_len = len(next(t))
#     if not all(len(l) == the_len for l in t):
#         print(f'{url} length check')
    detail_counts['counts'].append(num)
    detail_counts['urls'].append(url)
    return

In [508]:
# with futures.ThreadPoolExecutor() as getting:
#     getting.map(transform_details, detail_links)

In [473]:
# include = []
# exclude = []
# include.append('https://www.metacritic.com/game/pc/far-cry-5-lost-on-mars/details')
# exclude.append('https://www.metacritic.com/game/pc/painkiller-redemption/details')

In [ ]:
include

In [ ]:
exclude

In [521]:
for key in list(game_dict.keys())[:]:
    print(key,'length is', len(game_dict[key]), end = ", ")
#     newlist = game_dict[key][:-3]
#     print(f'New {key}','length is', len(newlist))
#     game_dict[key] = newlist

title length is 19318, release_date length is 19318, genre length is 19318, platforms length is 19318, developer length is 19318, esrb_rating length is 19318, ESRBs length is 19318, metascore length is 19318, userscore length is 19318, critic_reviews length is 19318, user_reviews length is 19318, num_players length is 19318, summary length is 19318, 

In [405]:
keys = list(game_dict.keys())

### Looking through failed detail pages 

In [476]:
for i, title in enumerate(game_dict['title']):
    if title=="Painkiller Redemption":
        print(i)

18838


In [490]:
idx = 18838
for key in keys[:]:
    print(key+': ', game_dict[key][idx])
#     if key=='title':
#         x = [game_dict[key][idx]]
#     else:
#         x.append(game_dict[key][idx])
# print(x)

title:  Zathura
release_date:  Nov  2, 2005
genre:  Action Adventure, Sci-Fi
platforms:  PlayStation 2
developer:  High Voltage Software
esrb_rating:  E10+
ESRBs:  Fantasy Violence
metascore:  43
userscore:  3.3
critic_reviews:  16
user_reviews:  6
num_players:  1 Player
summary:  The Zathura video game, based on the film of the same name, allows players to create their own high-action space adventures in a visually stunning 3D game environment. Based on the best-selling children's adventure book by Chris Van Allsburg ("Polar Express," "Jumanji"), Zathura follows two young brothers who find a mysterious board game that takes them on a wild ride through outer space. Meteor showers, out-of-control rocket-propelled robots and nasty alien lizards are among the obstacles they must overcome to get back to Earth. [2K Games]


In [266]:
# popped = {}

In [480]:
# idx = 18838
# for key in keys[:7]:
#     if key=='title':
#         x = [game_dict[key].pop(idx)]
        
#     else:
#         x.append(game_dict[key].pop(idx))

# popped['index'].append(idx)
# popped['contents'].append(x)
# pd.DataFrame(popped)   

,index,contents
0,9027,"[Yatagarasu: Attack on Cataclysm, Jul 7, 2015..."
1,11014,"[No Mans Land, Sep 15, 2017, Action, General, ..."
2,11433,"[Dracula Origin, Dec 19, 2008, Adventure, Gene..."
3,12096,"[Warlocks vs Shadows, TBA - Early Access, Role..."
4,12821,"[Area 51, Aug 8, 2019, Strategy, Action Adven..."
5,14850,"[P.I.D., Oct 13, 2020, Strategy, Adventure, Ge..."
6,16491,"[Naruto Shippuden: Shinobi Rumble, Feb 8, 201..."
7,18770,"[COPS 2170 The Power of Law, TBA, Strategy, Tu..."
8,18838,"[Painkiller Redemption, Feb 25, 2011, Action, ..."


In [336]:
summ = \
'''
[Xbox Live Arcade] The future of reality TV is Smash TV! It's the game show that asks: "How far would you go for a brand-new toaster?" You could win a VCR ... or you might lose your life! This new version of the original arcade game features online multiplayer co-op and versus modes. Listen to the audience cheer as you risk life and limb in room after carnage-filled room. Can you make it to the legendary Pleasure Dome? Cash and fabulous prizes could be yours - but only if you survive. And with the likes of Mr. Shrapnel, Mutoid Man, and Scarface out to get you, it's not likely. Good luck ... you're going to need it! [Xbox.com]
'''

In [337]:
summ = summ.strip()
summ

'[Xbox Live Arcade] The future of reality TV is Smash TV! It\'s the game show that asks: "How far would you go for a brand-new toaster?" You could win a VCR ... or you might lose your life! This new version of the original arcade game features online multiplayer co-op and versus modes. Listen to the audience cheer as you risk life and limb in room after carnage-filled room. Can you make it to the legendary Pleasure Dome? Cash and fabulous prizes could be yours - but only if you survive. And with the likes of Mr. Shrapnel, Mutoid Man, and Scarface out to get you, it\'s not likely. Good luck ... you\'re going to need it! [Xbox.com]'

In [338]:
summ in game_dict['summary']

False

In [14]:
games_df = pd.DataFrame(game_dict).drop_duplicates()
games_df

,title,release_date,genre,platforms,developer,esrb_rating,ESRBs,metascore,userscore,critic_reviews,user_reviews,num_players,summary
0,Burnout 3: Takedown,"Sep 7, 2004","Driving, Racing, Arcade",Xbox,Criterion Games,T,Mild Language Mild Violence,94,7.7,76,191.0,"1-2 Players, 8 Players Online",Burnout 3 challenges you to crash into (and th...
1,Jet Grind Radio,"Oct 30, 2000","Action, Platformer, 3D",Dreamcast,Smilebit,T,Animated Violence Mild Language,94,8.3,24,105.0,1 Player,"Join a graffiti crew, stamp your territory and..."
2,Metal Gear Solid 4: Guns of the Patriots,"Jun 12, 2008","Action Adventure, Modern, General, Modern",PlayStation 3,Kojima Productions,M,Blood Crude Humor Strong Language Suggestive T...,94,8.8,82,4231.0,"1 Player, 16 Players Online Up to 16 Players",Metal Gear Solid 4: Guns of the Patriots featu...
3,Tom Clancy's Splinter Cell Chaos Theory,"Mar 28, 2005","Action Adventure, Modern",Xbox,Ubisoft Montreal,M,Blood Strong Language Violence,94,9.1,70,233.0,"1-4 Players, 4 Players Online","As Sam Fisher, Third Echelon's most skilled Sp..."
4,Call of Duty: Modern Warfare 2,"Nov 10, 2009","Action, Shooter, Shooter, First-Person, Modern...",Xbox 360,Infinity Ward,M,Blood Drug Reference Intense Violence Language,94,6.6,100,3055.0,Up to 18 Players,Modern Warfare 2 continues the gripping and he...


In [510]:
# games_df.to_csv('metacritic_games_master.csv')

# Parse Review Pages

In [517]:
review_links = list(links_df['review_links'])

In [536]:
exclude_r = [link[:-7]+'user-reviews' for link in exclude]

In [540]:
len(exclude_r)

17

In [541]:
removes = []
for i, link in enumerate(review_links):
    if link in exclude_r:
        removes.append(i)

In [549]:
r_links = []
for i, link in enumerate(review_links):
    if i not in removes:
        r_links.append(link)

In [550]:
len(r_links)

19318

In [551]:
len(review_links)

19335

In [613]:
# review_counts = {'counts':[], 'urls':[]}

In [657]:
# failed_review_links = []
failed_review_links

['https://www.metacritic.com/game/pc/coast-guard/user-reviews']

In [606]:
def transform_reviews(url):  
    
    try:
        response = requests.get(url, headers = user_agent)
        soup = bs(response.text, 'html.parser')
    except Exception as e:
        print('\n', sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print(f'Trying Reviews again')
        sleep(10)
        response = requests.get(url, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        
        
    count = 1
    while soup.a==None and count<3:
        print('Error, Trying Reviews Again')
        sleep(10)
        response = requests.get(url, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        count += 1
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
    if soup.a==None:
        print('Trying Review last time')
        sleep(15)
        response = requests.get(url+'/robots.text', headers = user_agent)
        soup = bs(response.text, 'html.parser')
        print('Still None?:', soup.a==None)
        print('response code:', response.status_code)
        if soup.find('a')==None:
            print(f'Review Failed')
            print(url)
            failed_review_links.append(str(url))
            playsound('ding.mp3')
#             email(f'Failed to Connect to Page {page_num}')
            return
    
    
    if parse_reviews(soup, url)=='done':
        print(f' {url} went wrong')
        return
    num = next(iterate)
    review_counts['counts'].append(num)
    review_counts['urls'].append(url)
    return

In [ ]:
# with futures.ThreadPoolExecutor() as getting:
#     getting.map(transform_reviews, r_links[9000:])

In [658]:
# with futures.ThreadPoolExecutor() as getting:
#     getting.map(transform_reviews, failed_review_links)

In [726]:
reviews= pd.DataFrame(review_dict).drop_duplicates()
reviews['user'] = reviews.groupby('name').ngroup()
reviews = reviews[['user','name','game','rating','review']]
reviews

,user,name,game,rating,review
0,118429,ThomasR.,Combat Mission: Barbarossa to Berlin,9,Needs a brush-up on the graphics.
1,58304,JoaquinD.,Combat Mission: Barbarossa to Berlin,10,Perfect strategic and tactical WWII wargame. ...
2,78199,MichaelDorosh,Combat Mission: Barbarossa to Berlin,10,"Still the gold standard for WEGO, squad-based,..."
3,143673,dinin70,Combat Mission: Barbarossa to Berlin,9,I don't think it deserves 10/10 for a simple r...
4,14454,BlueFalcon,NHL 2002,6,"I was hyped about this release at the time, b..."
...,...,...,...,...,...
368885,16879,Buckshot,Ride to Hell: Retribution,0,It is just The guys who made this game should...
368886,126201,WayT101,Ride to Hell: Retribution,0,The 1% in this games title refers to this game...
368887,116801,TheKidWanted,Ride to Hell: Retribution,0,Ride to Hell:\r -Poor A.I-\r-Poor Storyline-\r...
368888,60903,Julzso23,Ride to Hell: Retribution,0,Possibly the worst game I have ever seen in a ...


In [700]:
# reviews.to_csv('metacritic_reviews_main.csv')

In [ ]:
# def get_reviews_per_game(startpage_link, next_page, end_page, game_num_start, game_num_end = 101): 
#     next_page = next_page
#     print(f'parsing page: {next_page-1}/{end_page}')
    
#     try:
#         response = requests.get(startpage_link, headers = user_agent)
#         soup = bs(response.text, 'html.parser')
#         current_page = int(soup.find('span', class_='page_num').text)
#     except:
#         playsound('ding.mp3')
    
# #     Loop through current page and extract links to game pages
#     game_num = game_num_start
#     for game in soup.find_all('a', attrs = {'class':'title'})[game_num - 1:game_num_end]:
#         game_link = url + game.get('href')
        
# #         Connects to game page summary
#         sleep(3)
#         try:
#             game_response = requests.get(game_link, headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
#         except:
#             playsound('ding.mp3')
        
# #         testing
        
        
#         count = 1
#         while game_soup.a==None and count<2:
#             print('Error, Trying GAME Again')
#             sleep(3)
#             game_response = requests.get(game_link, headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
#             count += 1
#             print('Still None?:', game_soup.a==None)
#             print('response code:', game_response.status_code)
        
#         if game_soup.a==None:
#             sleep(3)
#             game_response = requests.get(game_link+'/robots.text', headers = user_agent)
#             game_soup = bs(details_response.text, 'html.parser')
#             if game_soup.find('a')==None:
#                 print(f'Game {game_num} GAME Failed')
#                 playsound('ding.mp3')
#                 return
            
            
# #         get number of reviews
#         try:
#             if 
        
#             num_games['num_games'].append(int(game_soup.find_all('p', class_='see_all')[1].text.split()[2]))
#             num_games['title'].append(game_soup.find('h1').text)    
            
            
#         print(f'Game {game_num} completed')
#         game_num +=1
        
#     if next_page == (end_page+1):
#         find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
#         startpage_link = url + find_next_page.get('href')
#         print(f'Done, Next Start Page: {startpage_link}')
#         print('Copy and Paste the link to start_page or use startpage_link variable')
#         playsound('ding.mp3')
#         return startpage_link
#     if next_page>(end_page+1):
#         print('Increase End page')
#         playsound('ding.mp3')
#         return
#     if current_page>next_page:
#         print('Increase Next page')
#         playsound('ding.mp3')
#         return
#     sleep(3)
    
# #     Recurse through the following pages
#     find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
#     nextpage_link = url + find_next_page.get('href')
#     next_page += 1
#     get_reviews_per_game(nextpage_link, next_page, end_page, game_num_start = 1, game_num_end = 101)